In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# Get paths to the files
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.
df = pd.merge(df_ratings, df_ratings.groupby('user',as_index=False)['rating'].count(), on='user', suffixes=('', '_user_count'))
df = pd.merge(df, df_ratings.groupby('isbn',as_index=False)['rating'].count(), on='isbn', suffixes=('', '_isbn_count'))
df = pd.merge(df, df_books.drop('author', axis=1), on='isbn')
df = df.loc[(df['rating_user_count']>200) & (df['rating_isbn_count']>100)]
df = df.drop(['isbn', 'rating_user_count', 'rating_isbn_count'], axis=1)

In [5]:
# Pivot table
df = df.pivot_table(index='title',columns='user',values='rating').fillna(0)

In [6]:
# Show first 5 rows of the table
df.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Turn table into Compressed Sparse Row matrix
df_matrix = csr_matrix(df.values)

In [8]:
# Initialize NN model
neigh = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
# Fit the model
neigh.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [58]:
def get_recommends(book = ""):
  recommended_books = [book]
  recommendations = []
  distances, indices = neigh.kneighbors(df.loc[book].values.reshape(1, -1), n_neighbors = 6)
  distances = distances[0,:].tolist()
  distances.reverse()
  indices = indices[0,:].tolist()
  indices.reverse()
  for i in range(len(distances)-2):
    recommendations.append([df.index[indices[i]], distances[i]])
  recommended_books.append(recommendations)
  return recommended_books

In [60]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074]]]
You passed the challenge! 🎉🎉🎉🎉🎉
